In [ ]:
# For data generation
import sys
# !{sys.executable} -m pip install matplotlib --upgrade
import shutil
import os
import numpy as np
from glob import glob
import re
import pandas as pd
import pickle
#random
from time import perf_counter

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,root_mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
#Plotting
import seaborn as sns
sns.set_style()
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 200

import warnings
# warnings.filterwarnings("ignore", category=np.DeprecationWarning) 


from tqdm.auto import tqdm
# from tqdm.notebook import tqdm_notebook as tqdm

In [ ]:
with open(os.path.join(os.path.expanduser('~'),'DDCASPT2/drop.txt'),'r') as d:
    dropfeat = [i.replace('\n','') for i in d.readlines()]

In [ ]:
# with open('test_ind.pickle', 'rb') as handle:
#     test_ind = pickle.load(handle)

# with open('train_ind.pickle', 'rb') as handle:
#     train_ind = pickle.load(handle)
    
# print(len(train_ind),len(test_ind))    
# train_ind,test_ind = train_test_split(train_ind+test_ind, test_size=0.3, random_state=42)

radius_range_dirs = []
for i in glob('cluster/*.*'):
    try:
        radstr = float(os.path.basename(i))
        # if radstr>=1.4:
        radius_range_dirs.append(os.path.basename(i))
    except:
        continue

radius_range = sorted(radius_range_dirs)

train_ind,test_ind=radius_range[0::2],radius_range[1::2]

train_ind = list(map(float,train_ind))
test_ind = list(map(float,test_ind))
train_ind, test_ind = train_test_split(train_ind+test_ind, test_size=0.1, random_state=42)

In [ ]:
print(len(train_ind),len(test_ind))    

In [ ]:
rename = {'h$_{qq}^{0}$':'h$_{q}$',
'$(F_{q})_{0}$':'$F_{q}$',
'$(F_{q}^{\\text{SCF}})_{0}$':'$F_{q}^{\\text{SCF}}$',
'$(\\eta_{q})_{0}$':'$\\eta_{q}$',
'$(\\omega_{q})_{0}$':'$\\omega_{q}$',
'$(\\eta_{s})_{0}$':'$\\eta_{s}$',
'h$_{ss}^{0}$':'h$_{s}$',
'$(F_{s}^{\\text{SCF}})_{0}$':'$F_{s}^{\\text{SCF}}$',
'$(F_{s})_{0}$':'$F_{s}$',
'$(\\omega_{s})_{0}$':'$\\omega_{s}$',
'$(\\langle ss \\vert ss \\rangle)_{0}$':"$\\langle ss \\vert ss \\rangle$",
'$(\\langle qq \\vert qq \\rangle)_{0}$':"$\langle qq \\vert qq \\rangle$",
'h$_{pp}^{0}$': 'h$_{p}^{0}$',
'h$_{pp}^{1}$': 'h$_{p}^{1}$',
'h$_{pp}^{2}$': 'h$_{p}^{2}$',
'h$_{pp}^{3}$': 'h$_{p}^{3}$',
'h$_{rr}^{0}$': 'h$_{r}^{0}$',
'h$_{rr}^{1}$': 'h$_{r}^{1}$',
'h$_{rr}^{2}$': 'h$_{r}^{2}$',
'h$_{rr}^{3}$': 'h$_{r}^{3}$'}

In [ ]:
train = []
test = []



recover_train = []
recover_test = []

traincnt = 0
testcnt = 0

check_pairs = []
for k in train_ind:
    rad = f"{k:.2f}"
    traindf = pd.read_csv(os.path.join('cluster',rad,f'{rad}.csv'),compression='zip',index_col=0).drop(columns=dropfeat).rename(columns=rename)
    # traindf = traindf.loc[[i for i in traindf.index if i.split('_')[0]!=i.split('_')[1]]]
    # traindf = traindf[traindf['Pair_Energies'].abs()>=1e-6]
    check_pairs.append(traindf.reset_index()[['index','Pair_Energies']])
    train.append(traindf)
    recover_train.append((rad,traincnt,len(traindf)))
    traincnt+=1
        
for l in test_ind:
    rad = f"{l:.2f}"
    testdf = pd.read_csv(os.path.join('cluster',rad,f'{rad}.csv'),compression='zip',index_col=0).drop(columns=dropfeat).rename(columns=rename)
    # testdf = testdf.loc[[i for i in testdf.index if i.split('_')[0]!=i.split('_')[1]]]
    # testdf = testdf[testdf['Pair_Energies'].abs()>=1e-6]
    check_pairs.append(testdf.reset_index()[['index','Pair_Energies']])
    test.append(testdf)                    
    recover_test.append((rad,testcnt,len(testdf)))
    testcnt+=1




In [ ]:


# torch.tensor(, dtype=torch.float32).to(device)


X_train = np.array([i.drop(columns='Pair_Energies').values for i in train])
X_test = np.array([i.drop(columns='Pair_Energies').values for i in test])

y_train = np.array([i['Pair_Energies'].values for i in train])
y_test = np.array([i['Pair_Energies'].values for i in test])
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:


torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True


# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


# # Split data into training and testing sets
# # Standardize features and target
# scaler_X = StandardScaler()
# scaler_y = StandardScaler()

# X_train = scaler_X.fit_transform(X_train)
# X_test = scaler_X.transform(X_test)

# y_train = scaler_y.fit_transform(y_train.reshape(-1,1))
# y_test = scaler_y.transform(y_test.reshape(-1,1))

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

In [ ]:
X_train_tensor.shape,y_train_tensor.shape

In [ ]:
torch.sum(y_train_tensor,axis=1).shape

In [ ]:
# Define the regression model
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(102, 102 * 2),  # Input layer to first hidden layer
            nn.ReLU(),               # Activation function
            nn.Linear(102 * 2, 102 * 2),  # Hidden layers
            nn.ReLU(),
            nn.Linear(102 * 2, 102 * 2),
            nn.ReLU(),
            nn.Linear(102 * 2, 102 * 2),
            nn.ReLU(),
            nn.Linear(102 * 2, 102 * 2),
            nn.ReLU(),
            nn.Linear(102 * 2, 102 * 2),
            nn.ReLU(),
            nn.Linear(102 * 2, 102),  # Hidden to output layer
            nn.ReLU(),
            nn.Linear(102, 1)        # Output layer
        )

    def forward(self, x):
        return self.model(x)

# Initialize the model, loss function, and optimizer
model = RegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-6)

# Create DataLoader for training
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Train the model with mini-batches
num_epochs = 5000
losslist = []

for epoch in range(num_epochs):
    running_loss = 0
    for batch_X, batch_y in train_loader:
        # Move data to the appropriate device
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Forward pass
        outputs = model(batch_X).reshape(*batch_y.shape)
        # loss = criterion(torch.sum(outputs, axis=1), torch.sum(batch_y, axis=1))
        loss = criterion(outputs, batch_y)

        # Accumulate loss for the current batch
        running_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate average loss for the epoch and store it
    epoch_loss = running_loss / len(train_loader)
    losslist.append(epoch_loss)

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4e}')


In [ ]:
plt.plot(np.array(losslist))
plt.yscale('log')

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor)
    y_pred_train = model(X_train_tensor)

# De-standardize the predictions for interpretability
# y_pred_test = scaler_y.inverse_transform(y_pred.cpu().numpy())
# y_test_original = scaler_y.inverse_transform(y_test_tensor.cpu().numpy())

y_pred_test = y_pred_test.cpu().numpy().flatten()
y_test_original = y_test_tensor.cpu().numpy().flatten()

y_pred_train = y_pred_train.cpu().numpy().flatten()
y_train_original = y_train_tensor.cpu().numpy().flatten()


# Visualization
plt.scatter(y_test_original, y_pred_test, label=f"Test R2={r2_score(y_test_original, y_pred_test):.4f}")
plt.scatter(y_train_original, y_pred_train, label=f"Train R2={r2_score(y_train_original, y_pred_train):.4f}")
plt.legend()
plt.title("Regression Model Predictions")
plt.show()

In [ ]:
train_tss = np.sum((y_train-np.mean(y_train))**2)*1e3

In [ ]:
test_tss = np.sum((y_test-np.mean(y_test))**2)*1e3

In [ ]:
y_train = y_train.flatten()
y_pred_train = y_pred_train.flatten()
y_test = y_test.flatten()
y_pred_test = y_pred_test.flatten()

In [ ]:
example_train =np.array([(i, 1 - (i / train_tss)) for i in np.linspace(train_tss,0,100)])
fig, (ax1,ax2) = plt.subplots(1,2,sharey=True,figsize=(10,5))
ax1.plot(*example_train.T)
ax1.scatter(np.sum((y_train-y_pred_train)**2)*1e3,r2_score(y_train,y_pred_train),color='red',label=f"RSS={np.sum((y_train-y_pred_train)**2)*1e3:.4f}"+" mE$_{h}$\n"+f"TSS={np.sum((y_train-np.mean(y_train))**2)*1e3:.4f}"+" mE$_{h}$\n"+"R$^{2}$="+f"{r2_score(y_train,y_pred_train):.4f}")
ax1.set_xlabel('Residual Sum of Squares (mE$_{h}$)')
ax1.set_ylabel('R$^{2}$')
ax1.set_ylim(0,1)
ax1.set_xlim(0,train_tss)
ax1.legend()

example_test =np.array([(i, 1 - (i / test_tss)) for i in np.linspace(test_tss,0,100)])
ax2.plot(*example_test.T)
ax2.scatter(np.sum((y_test-y_pred_test)**2)*1e3,r2_score(y_test,y_pred_test),color='red',label=f"RSS={np.sum((y_test-y_pred_test)**2)*1e3:.4f}"+" mE$_{h}$\n"+f"TSS={np.sum((y_test-np.mean(y_test))**2)*1e3:.4f}"+" mE$_{h}$\n"+"R$^{2}$="+f"{r2_score(y_test,y_pred_test):.4f}")
ax2.set_xlabel('Residual Sum of Squares (mE$_{h}$)')
ax2.set_ylabel('R$^{2}$')
ax2.set_ylim(0,1)
ax2.set_xlim(0,test_tss)
ax2.legend()
plt.tight_layout()
#plt.savefig('images/RSS_vs_R2_original.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
pal = sns.color_palette("Paired")
font=12
plt.rc('font', size=font)          # controls default text sizes
plt.rc('axes', titlesize=font)     # fontsize of the axes title
plt.rc('axes', labelsize=font)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=font)    # fontsize of the tick labels
plt.rc('ytick', labelsize=font)    # fontsize of the tick labels
plt.rc('legend', fontsize=font)    # legend fontsize
plt.rc('figure', titlesize=font)  # fontsize of the figure title
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(13,6))
scale=1e-1
ax1.set_title('Train')
ax1.scatter(1e3*y_train,1e3*y_pred_train,label="R$^{2}$="+f'{r2_score(y_train,y_pred_train):.4f}\nMAE={1e3*mean_absolute_error(y_train,y_pred_train):.4f}'+" mE$_{h}$",color=pal[8],edgecolors='k')
ax1.plot(np.arange(-1e3,1e3),np.arange(-1e3,1e3),'k-')
LMIN,LMAX = -19, 10
ax1.set_xlim(LMIN,LMAX)
ax1.set_ylim(LMIN,LMAX)
ax1.set_xlabel('Calculated Pair-Energies (mE$_{h}$)')
ax1.set_ylabel('Predicted Pair-Energies (mE$_{h}$)')
ax1.legend()


ax2.set_title('Test')
ax2.scatter(1e3*y_test,1e3*y_pred_test,label="R$^{2}$="+f'{r2_score(y_test,y_pred_test):.4f}\nMAE={1e3*mean_absolute_error(y_test,y_pred_test):.4f}'+" mE$_{h}$",color=pal[9],edgecolors='k')
ax2.plot(np.arange(-1e3,1e3),np.arange(-1e3,1e3),'k-')
ax2.set_xlim(LMIN,LMAX)
ax2.set_ylim(LMIN,LMAX)
ax2.set_xlabel('Calculated Pair-Energies (mE$_{h}$)')
ax2.set_ylabel('Predicted Pair-Energies (mE$_{h}$)')
ax2.legend()

plt.tight_layout()
#plt.savefig('images/iron_VDZP_parity.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
testcntrcvr = 0
recover_test_list = []
recover_test_E2 = []
for j,k,l in recover_test:
    init_test = testcntrcvr
    testcntrcvr+=l
    recover_test_list.append((j,y_test[init_test:testcntrcvr],y_pred_test[init_test:testcntrcvr]))
    recover_test_E2.append((j,np.sum(y_test[init_test:testcntrcvr]),np.sum(y_pred_test[init_test:testcntrcvr])))


traincntrcvr = 0
recover_train_list = []
recover_train_E2 = []
for j,k,l in recover_train:
    init_train = traincntrcvr
    traincntrcvr+=l
    recover_train_list.append((j,y_train[init_train:traincntrcvr],y_pred_train[init_train:traincntrcvr]))
    recover_train_E2.append((j,np.sum(y_train[init_train:traincntrcvr]),np.sum(y_pred_train[init_train:traincntrcvr])))

In [ ]:
traincorrE2 = pd.DataFrame(recover_train_E2).rename(columns={0:'Radius ($\AA$)',1:'E$_{2}^{\text{Calculated}}$',2:'E$_{2}^{\text{Predicted}}$'}).set_index('Radius ($\AA$)')
#.astype({"basis set":str,'Radius ($\AA$)':float,'E$_{2}^{\text{Calculated}}$':float,'E$_{2}^{\text{Predicted}}$':float})
testcorrE2 = pd.DataFrame(recover_test_E2).rename(columns={0:'Radius ($\AA$)',1:'E$_{2}^{\text{Calculated}}$',2:'E$_{2}^{\text{Predicted}}$'}).set_index('Radius ($\AA$)')
#.astype({"basis set":str,'Radius ($\AA$)':float,'E$_{2}^{\text{Calculated}}$':float,'E$_{2}^{\text{Predicted}}$':float})



traincorrE2['CASPT2_E']=np.zeros(len(traincorrE2))
traincorrE2['CASSCF_E']=np.zeros(len(traincorrE2))
traincorrE2['E2']=np.zeros(len(traincorrE2))



for i in traincorrE2.index:
    energy_df = pd.read_excel(f"cluster/{i}/{i}_energies.xlsx",index_col=0)
    traincorrE2.loc[i,'CASPT2_E']=energy_df.loc['CASPT2_E'].values
    traincorrE2.loc[i,'CASSCF_E']=energy_df.loc['CASSCF_E'].values
    # traincorrE2.loc[i,'E2']=energy_df.loc['E2'].values

traincorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'] = traincorrE2['CASSCF_E']+traincorrE2['E$_{2}^{\text{Predicted}}$']

traincorrE2.rename(columns = {'CASPT2_E':'E$_{\text{CASPT2}}^{\text{Calculated}}$','CASSCF_E':'E$_{\text{CASSCF}}^{\text{Calculated}}$'},inplace=True)


testcorrE2['CASPT2_E']=np.zeros(len(testcorrE2))
testcorrE2['CASSCF_E']=np.zeros(len(testcorrE2))
testcorrE2['E2']=np.zeros(len(testcorrE2))



for i in testcorrE2.index:
    energy_df = pd.read_excel(f"cluster/{i}/{i}_energies.xlsx",index_col=0)
    testcorrE2.loc[i,'CASPT2_E']=energy_df.loc['CASPT2_E'].values
    testcorrE2.loc[i,'CASSCF_E']=energy_df.loc['CASSCF_E'].values
    # testcorrE2.loc[i,'E2']=energy_df.loc['E2'].values

testcorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'] = testcorrE2['CASSCF_E']+testcorrE2['E$_{2}^{\text{Predicted}}$']

testcorrE2.rename(columns = {'CASPT2_E':'E$_{\text{CASPT2}}^{\text{Calculated}}$','CASSCF_E':'E$_{\text{CASSCF}}^{\text{Calculated}}$'},inplace=True)

testcorrE2.reset_index(inplace=True)
traincorrE2.reset_index(inplace=True)

In [ ]:
traincaspt2melt = traincorrE2.reset_index().melt(id_vars=['Radius ($\AA$)'], value_vars=['E$_{\text{CASPT2}}^{\text{Calculated}}$','E$_{\text{CASPT2}}^{\text{Predicted}}$']).astype({'Radius ($\AA$)':float,'value':float})
testcaspt2melt = testcorrE2.reset_index().melt(id_vars=['Radius ($\AA$)'], value_vars=['E$_{\text{CASPT2}}^{\text{Calculated}}$','E$_{\text{CASPT2}}^{\text{Predicted}}$']).astype({'Radius ($\AA$)':float,'value':float})

In [ ]:
trueE2=pd.concat([traincorrE2.reset_index(),testcorrE2.reset_index()]).astype({'Radius ($\AA$)':float,'E$_{\text{CASPT2}}^{\text{Calculated}}$':float})

In [ ]:
testcorrE2['E$_{\text{CASPT2}}^{\text{Calculated}}$'].min()

In [ ]:
mean_absolute_error(testcorrE2['E$_{\text{CASPT2}}^{\text{Calculated}}$'],testcorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'])*1e3

In [ ]:
mean_absolute_error(traincorrE2['E$_{\text{CASPT2}}^{\text{Calculated}}$'],traincorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'])*1e3

In [ ]:
pairedcp=sns.color_palette('Paired')

In [ ]:
mean_absolute_error(traincorrE2['E$_{2}^{\text{Calculated}}$'],traincorrE2['E$_{2}^{\text{Predicted}}$'])*1e3

In [ ]:
mean_absolute_error(testcorrE2['E$_{2}^{\text{Calculated}}$'],testcorrE2['E$_{2}^{\text{Predicted}}$'])*1e3

In [ ]:
traincorrerr = {}

fig,((ax1,ax2),(ax3,ax4))=plt.subplots(2,2,figsize=(10,10),sharey=False)



pred_train,true_train = traincorrE2['E$_{2}^{\text{Predicted}}$'],traincorrE2['E$_{2}^{\text{Calculated}}$']
pred_test,true_test = testcorrE2['E$_{2}^{\text{Predicted}}$'],testcorrE2['E$_{2}^{\text{Calculated}}$']
testrad = testcorrE2['Radius ($\AA$)']    
train_r2_corr = r2_score(true_train,pred_train)
test_r2_corr = r2_score(true_test,pred_test)
test_mae_corr = 1e3*mean_absolute_error(true_test,pred_test)
train_mae_corr = 1e3*mean_absolute_error(true_train,pred_train)
print(pred_train.min(),true_train.min())
print(pred_train.max(),true_train.max())
traincorrerr = {"r2_train":train_r2_corr,"r2_test":test_r2_corr,"mae_train":train_mae_corr,"mae_test":test_mae_corr}
trainrad = traincorrE2['Radius ($\AA$)']
ax1.scatter(true_train,pred_train,label='R$^{2}$='+f'{train_r2_corr:.4f}\nMAE={train_mae_corr:.2e}'+" mE$_{h}$",color=pairedcp[8],edgecolors='k')

ax1.set_xlabel('Calculated CASPT2 Correlation Energy (E$_{h}$)')
ax1.set_ylabel('Predicted CASPT2 Correlation Energy (E$_{h}$)')
ax1.set_title('Train')
ax1.plot(np.arange(-1000,1000),np.arange(-1000,1000),'k')
ax1.legend(loc=4)


ax2.scatter(true_test,pred_test,label='R$^{2}$='+f'{test_r2_corr:.4f}\nMAE={test_mae_corr:.2e}'+" mE$_{h}$",color=pairedcp[9],edgecolors='k')
# ax2.plot(true_test,true_test,'k')
ax2.plot(np.arange(-1000,1000),np.arange(-1000,1000),'k')
ax2.set_xlabel('Calculated CASPT2 Correlation Energy (E$_{h}$)')
ax2.set_ylabel('Predicted CASPT2 Correlation Energy (E$_{h}$)')
ax2.set_title('Test')    
ax2.legend(loc=4)

bottom, top = -0.44, -0.36
ax1.set_xlim(bottom, top)
ax1.set_ylim(bottom, top)
ax2.set_xlim(bottom, top)
ax2.set_ylim(bottom, top)
ax1.set_xticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])
ax1.set_yticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])    
ax2.set_xticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])    
ax2.set_yticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])    

pred_train,true_train = traincorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'],traincorrE2['E$_{\text{CASPT2}}^{\text{Calculated}}$']
pred_test,true_test = testcorrE2['E$_{\text{CASPT2}}^{\text{Predicted}}$'],testcorrE2['E$_{\text{CASPT2}}^{\text{Calculated}}$']
testrad = testcorrE2['Radius ($\AA$)']    

print(pred_train.min(),true_train.min())
print(pred_train.max(),true_train.max())
traincorrerr = {"r2_train":train_r2_corr,"r2_test":test_r2_corr,"mae_train":train_mae_corr,"mae_test":test_mae_corr}
trainrad = traincorrE2['Radius ($\AA$)']
ax3.scatter(np.array(trainrad,dtype=float),pred_train,color=pairedcp[8],edgecolors='k')
ax3.plot(trueE2.sort_values(by='Radius ($\AA$)')['Radius ($\AA$)'],trueE2.sort_values(by='Radius ($\AA$)')['E$_{\text{CASPT2}}^{\text{Calculated}}$'],'k')
ax3.set_xlabel('Radius ($\AA$)')
ax3.set_ylabel('CASPT2 Energy (E$_{h}$)')
ax3.set_title('Train')
# ax3.legend()

ax4.scatter(np.array(testrad,dtype=float),pred_test,color=pairedcp[9],edgecolors='k')
ax4.plot(trueE2.sort_values(by='Radius ($\AA$)')['Radius ($\AA$)'],trueE2.sort_values(by='Radius ($\AA$)')['E$_{\text{CASPT2}}^{\text{Calculated}}$'],'k')
ax4.set_xlabel('Radius ($\AA$)')
ax4.set_ylabel('CASPT2 Energy (E$_{h}$)')
ax4.set_title('Test') 
# ax4.legend()

minpt2, maxpt2= pd.concat([true_train,true_test]).min(), pd.concat([true_train,true_test]).max()
pct=0.01
bufferbot, buffertop = minpt2*pct, maxpt2*pct




bottom, top = -1345.9, -1345.8
ax3.set_ylim(bottom,top)
ax3.set_yticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)]) 
ax4.set_ylim(bottom,top)
ax4.set_yticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)]) 

fig.suptitle("Fe(IV)O$^{2+}$ ANO-RCC-VDZP")

plt.subplots_adjust(wspace=0.05)
plt.tight_layout()
#plt.savefig(f'images/iron_VDZPCASPT2.png',dpi=300,bbox_inches='tight')
plt.show()    

In [ ]:
trainpairdf = pd.DataFrame(recover_train_list)
testpairdf = pd.DataFrame(recover_test_list)
pairerr={}

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5),sharey=True)

truetrainpair, predtrainpair = np.hstack(trainpairdf[1]),np.hstack(trainpairdf[2])
train_r2_pair = r2_score(truetrainpair, predtrainpair)
train_mae_pair = 1e3*mean_absolute_error(truetrainpair, predtrainpair)
ax1.scatter(truetrainpair, predtrainpair,label='R$^{2}$='+f'{train_r2_pair:.4f}\nMAE={train_mae_pair:.2e}'+" mE$_{h}$",color=pairedcp[8],edgecolors='k')
ax1.plot(np.arange(-1000,1000),np.arange(-1000,1000),'k')
ax1.set_xlabel('Calculated E$_2$ Energy (E$_{h}$)')
ax1.set_ylabel('Predicted E$_2$ Energy (E$_{h}$)')
ax1.set_title('Train')    
ax1.legend()  

truetestpair, predtestpair = np.hstack(testpairdf[1]),np.hstack(testpairdf[2])
test_r2_pair = r2_score(truetestpair, predtestpair)
test_mae_pair = 1e3*mean_absolute_error(truetestpair, predtestpair)
ax2.scatter(truetestpair, predtestpair,label='R$^{2}$='+f'{test_r2_pair:.4f}\nMAE={test_mae_pair:.2e}'+" mE$_{h}$",color=pairedcp[9],edgecolors='k')
ax2.plot(np.arange(-1000,1000),np.arange(-1000,1000),'k')
ax2.set_xlabel('Calculated E$_2$ Energy (E$_{h}$)')
ax2.set_ylabel('Predicted E$_2$ Energy (E$_{h}$)')
ax2.set_title('Test')    
ax2.legend()    

mincorr, maxcorr= np.hstack([truetrainpair,truetestpair]).min(), np.hstack([truetrainpair,truetestpair]).max()
pct=0.50
bufferbot, buffertop = mincorr*pct, maxcorr*pct

# if 'H' in n:
bottom, top = mincorr-buffertop, maxcorr+buffertop

# else:
#     bottom, top = mincorr+-0.1, maxcorr+0.05

ax1.set_xlim(bottom,top)
ax1.set_xticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])
ax1.set_ylim(bottom,top)
ax1.set_yticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])    
ax2.set_xlim(bottom,top)
ax2.set_xticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])    
ax2.set_ylim(bottom,top)
ax2.set_yticks(np.linspace(bottom,top,4),[f"{i:.2f}" for i in np.linspace(bottom,top,4)])    

fig.suptitle("Fe(IV)O$^{2+}$ ANO-RCC-VDZP")
plt.tight_layout()
#plt.savefig(f'images/iron_VDZPE2.png',dpi=300,bbox_inches='tight')    
plt.show()
pairerr = {"r2_train":train_r2_pair,"r2_test":test_r2_pair,"mae_train":train_mae_pair,"mae_test":test_mae_pair}

In [ ]:
stackeddata ={'Pair-Energies':{'Train':mean_absolute_error(truetrainpair, predtrainpair)*1e3,'Test':mean_absolute_error(truetestpair, predtestpair)*1e3},
"Correlation Energies":{'Train':mean_absolute_error(true_train,pred_train)*1e3,'Test':mean_absolute_error(pred_test,true_test)*1e3}}
#pd.DataFrame(stackeddata).to_excel("MAE_stacked.xlsx")